In [1]:
from __future__ import print_function
import pandas as pd
import logging
import numpy as np
from optparse import OptionParser
import sys, copy
from time import time
from random import randint
from math import log
#import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets import fetch_rcv1

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

In [2]:
#Code from https://stackoverflow.com/questions/8870261/how-to-split-text-without-spaces-into-list-of-words

# Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).
def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""

    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))

In [3]:
file_name = 'kkma_koreanonly_withsentencebreaker_1col.dat'
data_train = open(file_name).readlines()
out = open('rev_' + file_name,'w')
ENCODING = 'utf-8'
data_train_data = []
y_train = []
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

words = open("words-by-frequency.txt").read().split()
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))
maxword = max(len(x) for x in words)

for line_id,line in enumerate(data_train):
    items = line.split('\t')
    if len(items) == 2:
        out.write(items[0]+'\t')
        line = items[1].decode(ENCODING, 'ignore')
        ascii_mode = False
        tmp = ""
        #if(line_id%200 == 0):
        #    print(line)
        for i,c in enumerate(line):
            if(c!=' ' and is_ascii(c) and not ascii_mode):
                ascii_mode = True
                
            #if current character is not latin(means it is hangul) and there were some english characters before    
            if(not is_ascii(c) and ascii_mode):
                ascii_mode = False
             #   print(line_id,tmp)
                blocks = tmp.split(' ')
                for block in blocks:
                    block = block.lower()
                    spaced = infer_spaces(block)
                    for guess in spaced.split():
                        out.write(guess+' ')
                tmp=""
            if not ascii_mode:    
                out.write(c.encode('utf-8'))
            else:
                tmp += c.encode('utf-8')
            if c!=' ' and len(line)>i+1 and is_ascii(c) and not is_ascii(line[i+1]):
                #if(i%200 == 0): 
                #    print(i,c, line[i+1])
                out.write(' '.encode('utf-8'))
        out.write("\n")
    else:
        print("ERROR in train")
out.close()